In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

from matplotlib import pyplot as plt
import keras

# Any results you write to the current directory are saved as output.

In [ ]:
print(os.listdir("../input/pulsar-classification"))

In [ ]:
orig_dataset = pd.read_csv('../input/pulsar-classification/Train.csv')
print(orig_dataset.head())
final_dataset = pd.read_csv('../input/pulsar-classification/Test.csv')
print(final_dataset.head())

In [ ]:
orig_dataset['Pulsar'].value_counts().plot(kind = 'pie', title = 'Count (Pulsar)')
plt.show()

In [ ]:
train_X = np.array(orig_dataset.iloc[:, 0:-1])
train_Y = np.array(orig_dataset.iloc[:, -1])
final_X = np.array(final_dataset.iloc[:,:])

In [ ]:
from imblearn.over_sampling import RandomOverSampler

X = train_X.copy()
Y = train_Y.copy()

ros = RandomOverSampler()
X, Y = ros.fit_sample(X, Y)

Y = Y.reshape(-1,1)

print(X.shape)
print(Y.shape)

In [ ]:
train_X=X
train_Y=Y
print(train_X.shape)
print(train_Y.shape)
print(final_X.shape)

In [ ]:
from imblearn.over_sampling import RandomOverSampler

X = train_X.copy()
Y = train_Y.copy()

ros = RandomOverSampler()
X, Y = ros.fit_sample(X, Y)

Y = Y.reshape(-1,1)

print(X.shape)
print(Y.shape)

In [ ]:
df = pd.DataFrame(Y)
df.columns=['Pulsar']
df.Pulsar.value_counts().plot(kind='pie', title = 'Count')
plt.show()
train_X = X
train_Y = Y

In [ ]:
#splitting the dataset
from sklearn.model_selection import StratifiedShuffleSplit

stratified_split = StratifiedShuffleSplit(n_splits = 2, random_state = 0, test_size = 0.05)

for train_index, test_index in stratified_split.split(train_X, train_Y):
    X_train, X_test = train_X[train_index], train_X[test_index]
    Y_train, Y_test = train_Y[train_index], train_Y[test_index]

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

std_sclr = StandardScaler()
X_train = std_sclr.fit_transform(X_train)
X_test = std_sclr.fit_transform(X_test)

In [ ]:
X_train = X_train.T
Y_train = Y_train.reshape(-1, 1).T

X_test = X_test.T
Y_test = Y_test.reshape(-1, 1).T

print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)

print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)

In [ ]:
new_x_train = X_train.copy().T
new_y_train = Y_train.copy().T
new_x_test = X_test.copy().T
new_y_test = Y_test.copy().T

In [ ]:
print(new_x_train.shape)
print(new_y_train.shape)
print(new_x_test.shape)
print(new_y_test.shape)

In [ ]:
def build_keras_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(4, activation = 'relu', input_shape = (new_x_train.shape[1],)))
    
    model.add(keras.layers.Dense(128, activation = 'relu',))
    
    model.add(keras.layers.Dense(128, activation = 'relu',))
    
    model.add(keras.layers.Dense(64, activation = 'relu',))
    
    model.add(keras.layers.Dense(64, activation = 'relu',))
    
    model.add(keras.layers.Dense(32, activation = 'relu',))
    
    model.add(keras.layers.Dense(32, activation = 'relu'))
    
    model.add(keras.layers.Dense(1, activation = 'sigmoid'))
    
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])
    
    return model

In [ ]:
model = build_keras_model()
model.summary()

In [ ]:
callbacks = [keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3, restore_best_weights = True)]

In [ ]:
history = model.fit(new_x_train, new_y_train, epochs = 1000, callbacks = callbacks)

In [ ]:
new_preds = model.predict(new_x_test)

def find_accuracy(prediction_array, data_array):
    l=0
    for i in prediction_array:
        if i>=0.5:
            prediction_array[l]=1
        else:
            prediction_array[l]=0
        l=l+1
    j=0
    
    j = np.sum(prediction_array==data_array)
    print(j)
    return (j*100)/(int(prediction_array.shape[0]))
    

In [ ]:
print(new_preds.shape)
print(new_y_test.shape)
print("Accuracy: " + str(find_accuracy(new_preds, new_y_test)))

In [ ]:
print(os.listdir("../input/predicting-a-pulsar-star"))

In [ ]:
#testing
test_dataset = pd.read_csv('../input/predicting-a-pulsar-star/pulsar_stars.csv')
test_dataset.shape

In [ ]:
test_x = np.array(test_dataset.iloc[:,0:-1])
test_y = np.array(test_dataset.iloc[:,-1])

In [ ]:
print(test_x.shape)
print(test_y.shape)

In [ ]:
test_pred = model.predict(test_x)

In [ ]:
print(test_pred.shape)
test_y = test_y.reshape(-1,1)
print(test_y.shape)
print("Accuracy : " + str(find_accuracy(test_pred, test_y)))

In [ ]:
final = model.predict(final_X)
l=0
for i in final:
    if i>=0.5:
        final[l]=1
    else:
        final[l]=0
f = np.zeros((final.shape[0]))
p=0
while p<5370:
    f[p]=p+1
    p=p+1
print(f.shape)
f=f.reshape(-1,1)
print(f.shape)
print(final.shape)


In [ ]:
s = np.concatenate((f,final), axis=1)
pd.DataFrame(s).to_csv("Predictions.csv", header=['Id', 'Predictions'], index=None)
print("Done")
    